## Model Compilation Jupyter Notebook Example

This notebook shows the example of model compilation using edgeai-benchmark.

This script uses TIDL to compile a model and output in a format that edgeai-sdk can understand.

edgeai_benchmark is a python package provided in edgeai-benchmark that provides several functions to assist model compilation.

In [1]:
import os
import tempfile
import argparse
import cv2
import yaml
from edgeai_benchmark import *

In [2]:
# the cwd must be the root of the respository
if os.path.split(os.getcwd())[-1] in ('scripts', 'tutorials'):
    os.chdir('../')
#
print(os.environ['TIDL_TOOLS_PATH'])
print(f"\nINFO: current dir is: {os.getcwd()}")

/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/tools/AM62A/tidl_tools

INFO: current dir is: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark


#### Create a temporary directory. 

This is were the compiled artifacts will be stored.

In [3]:
modelartifacts_tempdir_name = './work_dirs_custom'
modelartifacts_custom = os.path.join(modelartifacts_tempdir_name, 'modelartifacts')

#### Read settings from settings_import_on_pc.yaml

Modify the settings as necessary in the constructor of settings.ConfigSettings()

In [4]:
settings = config_settings.CustomConfigSettings('./settings_import_on_pc.yaml',
                modelartifacts_path=modelartifacts_custom,
                calibration_frames=10, calibration_iterations=10, num_frames=100)

work_dir = os.path.join(settings.modelartifacts_path, f'{settings.tensor_bits}bits')
print(f'INFO: work_dir = {work_dir}')

INFO: work_dir = ./work_dirs_custom/modelartifacts/8bits


#### Create Dataset Reader classes

Change the dataset paths according to your dataset location

In [5]:
dataset_calib_cfg = dict(
    path=f'{settings.datasets_path}/coco',
    split='val2017',
    shuffle=True,
    num_frames=min(settings.calibration_frames,5000),
    name='coco'
)

# dataset parameters for actual inference
dataset_val_cfg = dict(
    path=f'{settings.datasets_path}/coco',
    split='val2017',
    shuffle=False, # can be set to True as well, if needed
    num_frames=min(settings.num_frames,5000),
    name='coco'
)

calib_dataset = datasets.COCODetection(**dataset_calib_cfg, download=True)
val_dataset = datasets.COCODetection(**dataset_val_cfg, download=True)


INFO:20250226-123321: dataset exists - will reuse - ./dependencies/datasets/coco
loading annotations into memory...
Done (t=0.48s)
creating index...
index created!

INFO:20250226-123323: dataset exists - will reuse - ./dependencies/datasets/coco
loading annotations into memory...
Done (t=0.55s)
creating index...
index created!


#### Session runtime_options

The default runtime_options can be overriden by passing a runtime_options dict to this function

In [6]:
# choose one session_name depending on the model type
# tflitert for tflite models, onnxrt for onnx models, tvmdlr for mxnet models.
session_name = constants.SESSION_NAME_TFLITERT
#session_name = constants.SESSION_NAME_ONNXRT
#session_name = constants.SESSION_NAME_TVMDLR

session_type = settings.get_session_type(session_name)
runtime_options = settings.get_runtime_options(session_name, is_qat=False)

print(session_type)
print(runtime_options)

<class 'edgeai_benchmark.sessions.tflitert_session.TFLiteRTSession'>
{'tensor_bits': 8, 'accuracy_level': 1, 'debug_level': 0, 'inference_mode': 0, 'advanced_options:high_resolution_optimization': 0, 'advanced_options:pre_batchnorm_fold': 1, 'advanced_options:calibration_frames': 10, 'advanced_options:calibration_iterations': 10, 'advanced_options:quantization_scale_type': None, 'advanced_options:activation_clipping': 1, 'advanced_options:weight_clipping': 1, 'advanced_options:bias_calibration': 1, 'advanced_options:output_feature_16bit_names_list': '', 'advanced_options:params_16bit_names_list': '', 'advanced_options:add_data_convert_ops': 3, 'advanced_options:c7x_firmware_version': '10_01_04_00', 'ti_internal_nc_flag': 83886080}


In [7]:
preproc_transforms = preprocess.PreProcessTransforms(settings)
postproc_transforms = postprocess.PostProcessTransforms(settings)

# these session cfgs also has some default input mean and scale. 
# if your model needs a difference mean and scale, update the session cfg dict being used with those values
onnx_session_cfg = sessions.get_onnx_session_cfg(settings, work_dir=work_dir)
tflite_session_cfg = sessions.get_tflite_session_cfg(settings, work_dir=work_dir)

#### Create pipeline_configs

pipeline_configs is nothing but a dict with the various model configs that we want to compile

In [8]:
pipeline_configs = {
    'od-mlpefmnv1': dict(
        task_type='detection',
        calibration_dataset=calib_dataset,
        input_dataset=val_dataset,
        preprocess=preproc_transforms.get_transform_tflite((300,300), (300,300), backend='cv2'),
        session=session_type(**tflite_session_cfg,
            runtime_options=runtime_options,
            model_path=f'{settings.models_path}/vision/detection/coco/mlperf/ssd_mobilenet_v1_coco_20180128.tflite'),
        postprocess=postproc_transforms.get_transform_detection_tflite(),
        metric=dict(label_offset_pred=datasets.coco_det_label_offset_90to90()),
        model_info=dict(metric_reference={'accuracy_ap[.5:.95]%':23.0})
    )
}
print(f"\nINFO: pipeline_configs={pipeline_configs}")


INFO: pipeline_configs={'od-mlpefmnv1': {'task_type': 'detection', 'calibration_dataset': <edgeai_benchmark.datasets.coco_det.COCODetection object at 0x7fbb7fcf73d0>, 'input_dataset': <edgeai_benchmark.datasets.coco_det.COCODetection object at 0x7fbbdd99e8f0>, 'preprocess': <edgeai_benchmark.preprocess.PreProcessTransforms object at 0x7fbb7fcf7130>, 'session': <edgeai_benchmark.sessions.tflitert_session.TFLiteRTSession object at 0x7fbb7fcf6980>, 'postprocess': <edgeai_benchmark.postprocess.PostProcessTransforms object at 0x7fbb7fcf6890>, 'metric': {'label_offset_pred': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 5

#### Model Compilation

This will take a few minutes. Please be patient...

In [9]:
# run the model compliation/import and inference
interfaces.run_benchmark_config(settings, work_dir, pipeline_configs)
run_dir = list(pipeline_configs.values())[0]['session'].get_run_dir()
print(f"\nINFO: compiled artifacts is in: {run_dir}")

# print the result:
with open(os.path.join(run_dir, "result.yaml")) as fp:
    result_yaml = yaml.safe_load(fp)
    result_dict = result_yaml['result']
    print(f"INFO: result - {result_dict}")
#

INFO: number of configs: 1


TASKS TOTAL=1, NUM_RUNNING=0:   0%|                                                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s, postfix={'RUNNING': [], 'COMPLETED': []}]


INFO:20250226-123324: starting - od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite

SUCCESS:20250226-123324: found results - {'infer_path': 'od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite', 'accuracy_ap[.5:.95]%': 29.477315, 'accuracy_ap50%': 43.220232, 'num_subgraphs': 0}



TASKS TOTAL=1, NUM_RUNNING=0:   0%|                                                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s, postfix={'RUNNING': [], 'COMPLETED': []}]


INFO:20250226-123325: starting - od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite

SUCCESS:20250226-123325: found results - {'infer_path': 'od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite', 'accuracy_ap[.5:.95]%': 29.477315, 'accuracy_ap50%': 43.220232, 'num_subgraphs': 0}



TASKS TOTAL=1, NUM_RUNNING=0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/it, postfix={'RUNNING': [], 'COMPLETED': ['od-mlpefmnv1']}]


INFO: compiled artifacts is in: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite
INFO: result - {'infer_path': 'od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite', 'accuracy_ap[.5:.95]%': 29.477315, 'accuracy_ap50%': 43.220232, 'num_subgraphs': 0}


#### Package artifacts

Package the artifacts into a .tar.gz file, keeping only the necessary files for inference.

In [10]:
print()
out_dir = f'{work_dir}_package'
interfaces.package_artifacts(settings, work_dir, out_dir)


INFO: packaging artifacts to ./work_dirs_custom/modelartifacts/8bits_package please wait...
SUCCESS:20250226-123327: finished packaging - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite


#### Download

Download the packaged .tar.gz artifact

TODO: add a download link here, that the user can click to download the packaged artifact .tar.gz file.

In [11]:
print(f'\nINFO: download the atricats files from the folder: {out_dir}')
print(os.listdir(out_dir))


INFO: download the atricats files from the folder: ./work_dirs_custom/modelartifacts/8bits_package
['artifacts.csv', 'artifacts.yaml', 'od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite.tar.gz', 'extract.sh']
